In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import hashlib
from google.colab import drive

drive.mount('/content/drive')

headers = {
    "user-agent": "Chrome/91.0.4472.101"
}

job2_list = ['Java-Developer','Testing','DevOps-Engineer','Python-Developer','Web-Designing','HR','Hadoop','Blockchain','ETL-Developer','Operations-Manager','Data-Science','Sales','Mechanical-Engineer',
 'Arts','Database','Electrical-Engineering','Health-and-fitness','PMO','Business-Analyst','DotNet-Developer','Automation-Testing','Network-Security-Engineer','SAP-Developer','Civil-Engineer','Advocate']

job_list = ['Java Developer','Testing','DevOps Engineer','Python Developer','Web Designing','HR','Hadoop','Blockchain','ETL Developer','Operations Manager','Data Science','Sales','Mechanical Engineer',
'Arts','Database','Electrical Engineering','Health and fitness','PMO','Business Analyst','DotNet Developer','Automation Testing','Network Security Engineer','SAP Developer','Civil Engineer','Advocate']
resume_links = pd.DataFrame()
category = []
link = []
for i in range(0,len(job2_list)):
  job = job2_list[i]
  JOB2 = job_list[i]
  JOB = job.lower()
  print(JOB)
  for i in range(1,4):
    PAGE = str(i)
    URL = "https://www.livecareer.com/resume-search/search?jt="+ JOB + "&bg=85&eg=100&comp=&mod=&pg="+ PAGE
    page = requests.get(URL, headers=headers, timeout=20).content
    soup = BeautifulSoup(page, "html.parser")
    prettified_html = soup.prettify()
    a_tags = soup.find_all('a', class_='sc-1dzblrg-0')
    for a_tag in a_tags:
      href_value = a_tag.get('href')
      category.append(JOB2)
      link.append(href_value)

resume_links["Category"] = category
resume_links["link"] = link

def extract_next_div_text_after(html_content, section_title):
    soup = BeautifulSoup(html_content, 'html.parser')

    start_section = soup.find('div', class_='sectiontitle', string=lambda text: section_title in text)
    text_content = ""
    if start_section:
        next_div = start_section.find_next('div')
        if next_div:
            text_content = next_div.get_text(separator='\n', strip=True)
    start_section = soup.find('div', class_='sectiontitle', string=lambda text: "Experience" in text)
    if start_section:
        next_div = start_section.find_next('div')

        if next_div:
            text_content = text_content + next_div.get_text(separator='\n', strip=True)

    return text_content if len(text_content) > 0 else None

headers = {
    "user-agent": "Chrome/91.0.4472.101"
}

resume_links["Resume"] = ""

print(resume_links.head())

for i in range(resume_links.shape[0]):
    url = 'https://www.livecareer.com' + resume_links.link[i]
    page = requests.get(url, headers=headers, timeout=10).content
    section_title = "Skills"
    div = extract_next_div_text_after(page, section_title)
    if div:
      resume_links.Resume[i] = div

In [ ]:
resume_links = resume_links.drop(columns=['link'])
resume_links = resume_links[resume_links['Resume'].notna() & (resume_links['Resume'] != '')]
resume_links.to_csv('/content/drive/MyDrive/updated_resume_data.csv', index=False)